In [140]:
#we knows how to make a tools for the llm model using the ai agents lets see how they are called by llm 



In [141]:
#firstly create a tool 
from langchain_core.tools import tool 
@tool
def  multiply(a: int,b: int)-> int:
    '''Multiply 2 numbers'''
    return a*b 


In [142]:
from langchain_core.messages import HumanMessage 
from langchain_groq import ChatGroq


In [143]:
#let first make a llm instace
import os 
from dotenv import load_dotenv 
load_dotenv()
groq_api_key=os.environ['GROQ_API_KEY']
llm=ChatGroq(model='Llama3-70b-8192')


In [144]:
llm.invoke('hi') #so our llm is perfectly working fine 


AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.071428571, 'prompt_time': 0.000125138, 'queue_time': 0.052944343000000005, 'total_time': 0.071553709}, 'model_name': 'Llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--c0163e82-480c-4c4b-8751-656f8b440925-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [145]:
llm_with_tools=llm.bind_tools([multiply]) #currently we are passing a single tool but we can also pass multiple tools 

In [146]:
messages=[]
query='what is multiplication of 50 and 55'

In [147]:
messages.append(HumanMessage(query))

In [148]:
ai_response=llm_with_tools.invoke(messages)
messages.append(ai_response)
print(ai_response)

content='' additional_kwargs={'tool_calls': [{'id': 'xbx4qa795', 'function': {'arguments': '{"a":50,"b":55}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 901, 'total_tokens': 937, 'completion_time': 0.104458165, 'prompt_time': 0.029164582, 'queue_time': 0.054052488, 'total_time': 0.133622747}, 'model_name': 'Llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d114d084-a219-431b-8868-746b76c08c56-0' tool_calls=[{'name': 'multiply', 'args': {'a': 50, 'b': 55}, 'id': 'xbx4qa795', 'type': 'tool_call'}] usage_metadata={'input_tokens': 901, 'output_tokens': 36, 'total_tokens': 937}


In [149]:
tool_call=ai_response.tool_calls[0]

In [150]:
tool_call

{'name': 'multiply',
 'args': {'a': 50, 'b': 55},
 'id': 'xbx4qa795',
 'type': 'tool_call'}

In [151]:
tool_msg=multiply.invoke(tool_call)
messages.append(tool_msg)
print(tool_msg) #the above return a tool message 

content='2750' name='multiply' tool_call_id='xbx4qa795'


In [152]:
#and we also can call the function usign the args value only 
multiply.invoke(tool_call['args']) #but this will return the result only

2750

In [153]:
messages

[HumanMessage(content='what is multiplication of 50 and 55', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'xbx4qa795', 'function': {'arguments': '{"a":50,"b":55}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 901, 'total_tokens': 937, 'completion_time': 0.104458165, 'prompt_time': 0.029164582, 'queue_time': 0.054052488, 'total_time': 0.133622747}, 'model_name': 'Llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d114d084-a219-431b-8868-746b76c08c56-0', tool_calls=[{'name': 'multiply', 'args': {'a': 50, 'b': 55}, 'id': 'xbx4qa795', 'type': 'tool_call'}], usage_metadata={'input_tokens': 901, 'output_tokens': 36, 'total_tokens': 937}),
 ToolMessage(content='2750', name='multiply', tool_call_id='xbx4qa795')]

In [181]:
#now lets create 2 tools and attach them to the llm
from langchain.tools import tool  
from typing import Annotated 
from langchain_core.tools import InjectedToolArg  
@tool 
def Multiply(a: float,b: float)-> float: 
    """Multiply 2 numbers """
    return a*b 
# @tool   #if we want to pass the returned value from teh multiplication to the addition then perform addition 
# def Addition(a: Annotated[float,InjectedToolArg],b: float )-> float:
#     """Addition of 2 numbers """
#     return a+b 

#if not then following tool is correct 
@tool 
def Addition(a: float,b: float)-> float:
    """Addition of 2 given numbers"""
    return a+b



In [182]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='Llama3-70b-8192')
# llm.invoke('hi')

In [183]:
#working fine lets add tool to it 
#but here we have to make a instance for the usage of the tools
llm_with_tools=llm.bind_tools([Multiply,Addition])


In [184]:
msg=[]
query=HumanMessage('what is 3 multiply by 8 and 3993 addition with 878') #if we have to use this then we have
#to remove the annotated attribute from the tool of addition 's arugments 

# query2=HumanMessage('multiply 98 with 544 and then add 5758 with that multiplied value')
msg.append(query)

In [185]:
#not lets invoke the llm model with the given query 
ai_msg=llm_with_tools.invoke(msg)

In [186]:
#lets append this to the msg 
msg.append(ai_msg)

In [189]:
ai_msg.tool_calls  #here we have to extract both tool calls and then return the response to the llm 
#tool call look like this

# [{'name': 'Multiply',
#   'args': {'a': 98, 'b': 544},
#   'id': 'qa26x3c3v',
#   'type': 'tool_call'},
#  {'name': 'Addition',
#   'args': {'a': 0, 'b': 5758},
#   'id': '4nns5nj88',
#   'type': 'tool_call'}] #here in the tool_calls we can see it is taking the value as 0 
#but it should calculate teh multiplication first and then shold pass it ot the addition 
#so we have to modify the function in such a way that the llm stick to the logic never done thing self without tools



#and now for our work like first multiplication then the addition for this we ahve to add the returned value to 
#the addition function's arguments


[{'name': 'Multiply',
  'args': {'a': 3, 'b': 8},
  'id': 'dq528csq6',
  'type': 'tool_call'},
 {'name': 'Addition',
  'args': {'a': 3993, 'b': 878},
  'id': 'rwyva4868',
  'type': 'tool_call'}]

In [190]:
# multiplication=...

# for tool_call in ai_msg.tool_calls:

#     if tool_call['name']=='Multiply':
#        tool_msg1=Multiply.invoke(tool_call)
#        msg.append(tool_msg1) 
#        print(tool_msg1)
#        #we have to store teh value that is returned from the multiply and then pass it to the addition
#        multiplication=tool_msg1.content
#        print(multiplication) #so we are getting the values ok now pass it to the addition invocation
       
#        #we have to get the multiplied value and store it in the variable
#     if tool_call['name']=='Addition':
#         print(tool_call)
#         tool_call['args']['a']=multiplication
#         print(tool_call['args'])
        
#         tool_msg2=Addition.invoke(tool_call)
#         msg.append(tool_msg2)
        
#         print(tool_msg2)
    
#NOTE THE ABOVE LOGIS IS USED IF WE WANT TO ADD THE FUNCITONAILITY 
# LIKE: FIRST CALCULATE THE MULTIPLICATION THEN PASS IT TO THE ADDITION AND THEN ADD A VALUE IN IT 
#IF NOT THEN THE FOLLOWIG LOGIC FOR THE PARALLEL OPERATION 
for tool_call in ai_msg.tool_calls:
    if tool_call['name']=='Multiply':
        tool_msg1=Multiply.invoke(tool_call)
        msg.append(tool_msg1)
        print(tool_msg1)
    if tool_call['name']=='Addition':
        tool_msg2=Addition.invoke(tool_call)
        msg.append(tool_msg2)
        print(tool_msg2)

content='24.0' name='Multiply' tool_call_id='dq528csq6'
content='4871.0' name='Addition' tool_call_id='rwyva4868'


In [176]:
#lets pass it to the llm and then it will generate the reponse
llm_with_tools.invoke(msg)

AIMessage(content='The result of the multiplication is 24.0 and the result of the addition is 4871.0.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 1158, 'total_tokens': 1182, 'completion_time': 0.118739809, 'prompt_time': 0.045250062, 'queue_time': 0.06689783799999999, 'total_time': 0.163989871}, 'model_name': 'Llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--cecedfd4-f3b6-432e-a9b1-59cb5a86e8a0-0', usage_metadata={'input_tokens': 1158, 'output_tokens': 24, 'total_tokens': 1182})

In [ ]:
#lets make an another case also where we have to perform task in the series by changing teh query 
#from old to query=multiply 98 with 544 and add 8788 with it and return its value 




#NOTE THE ABOVE CODE IS NOT A AI AGENT HERE WE ARE INVOKING THE TOOLS
#BUT THE AI AGENT CAN DO THIS BY ITSELF THAT'S IS TEH BIG DIFFERNCE
